In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = None

In [ ]:
gl=pd.read_csv("/data/GameLogs.csv", encoding='utf-16')

In [ ]:
gl[(gl["HomeTeam"]=="CHA")&(pd.to_datetime(gl['Date'],format='%Y%m%d').dt.year==2019)]

,Date,NumberofGames,DayofWeek,VisitingTeam,VisitingTeamLeague,VisitingTeamGameNumber,HomeTeam,HomeTeamLeague,HomeTeamGameNumber,VistingTeamScore,HomeTeamScore,NumberofOuts,DayNight,Completition_Information,Forfeit_Information,Protest_Information,BallParkID,Attendance,LengthofGame,VisitingTeam_LineScore,HomeTeam_LineScore,VisitingTeamOffense_AtBats,VisitingTeamOffense_Hits,VisitingTeamOffense_Doubles,VisitingTeamOffense_Triples,VisitingTeamOffense_Homeruns,VisitingTeamOffense_RBIs,VisitingTeamOffense_SacrificeHits,VisitingTeamOffense_SacrificeFlies,VisitingTeamOffense_HitbyPitch,VisitingTeamOffense_Walks,VisitingTeamOffense_IntentionalWalks,VisitingTeamOffense_Strickouts,VisitingTeamOffense_StolenBases,VisitingTeamOffense_CaughtStealing,VisitingTeamOffense_GroundedintoDoublePlay,VisitingTeamOffense_AwardedFirstonCatcherInterference,VisitingTeamOffense_LeftOnBase,VisitingTeamPitchers_PitchersUsed,VisitingTeamPitchers_IndividualEarnedRuns,VisitingTeamPitchers_TeamEarnedRuns,VisitingTeamPitchers_WildPitches,VisitingTeamPitchers_Balks,VisitingTeamDefense_PutOuts,VisitingTeamDefense_Assists,VisitingTeamDefense_Errors,VisitingTeamDefense_PassedBalls,VisitingTeamDefense_DoublePlays,VisitingTeamDefense_TriplePlays,HomeTeamOffense_AtBats,HomeTeamOffense_Hits,HomeTeamOffense_Doubles,HomeTeamOffense_Triples,HomeTeamOffense_Homeruns,HomeTeamOffense_RBIs,HomeTeamOffense_SacrificeHits,HomeTeamOffense_SacrificeFlies,HomeTeamOffense_HitbyPitch,HomeTeamOffense_Walks,HomeTeamOffense_IntentionalWalks,HomeTeamOffense_Strickouts,HomeTeamOffense_StolenBases,HomeTeamOffense_CaughtStealing,HomeTeamOffense_GroundedintoDoublePlay,HomeTeamOffense_AwardedFirstonCatcherInterference,HomeTeamOffense_LeftOnBase,HomeTeamPitchers_PitchersUsed,HomeTeamPitchers_IndividualEarnedRuns,HomeTeamPitchers_TeamEarnedRuns,HomeTeamPitchers_WildPitches,HomeTeamPitchers_Balks,HomeTeamDefense_PutOuts,HomeTeamDefense_Assists,HomeTeamDefense_Errors,HomeTeamDefense_PassedBalls,HomeTeamDefense_DoublePlays,HomeTeamDefense_TriplePlays,HomePlateUmp_ID,HomePlateUmp_Name,1BUmp_ID,1BUmp_Name,2BUmp_ID,2BUmp_Name,3BUmp_ID,3BUmp_Name,LFUmp_ID,LFUmp_Name,RFUmp_ID,RFUmp_Name,VisitingTeamManager_ID,VisitingTeamManager_Name,HomeTeamManager_ID,HomeTeamManager_Name,WinningPitcher_ID,WinningPitcher_Name,LosingPitcher_ID,LosingPitcher_Name,SavingPitcher_ID,SavingPitcher_Name,GameWinningRBIBatter_ID,GameWinningRBIBatter_Name,VisitingTeam_StartingPitcher_ID,VisitingTeam_StartingPitcher_Name,HomeTeam_StartingPitcher_ID,HomeTeam_StartingPitcher_Name,VisitingTeam_Player1_ID,VisitingTeam_Player1_Name,VisitingTeam_Player1_Position,VisitingTeam_Player2_ID,VisitingTeam_Player2_Name,VisitingTeam_Player2_Position,VisitingTeam_Player3_ID,VisitingTeam_Player3_Name,VisitingTeam_Player3_Position,VisitingTeam_Player4_ID,VisitingTeam_Player4_Name,VisitingTeam_Player4_Position,VisitingTeam_Player5_ID,VisitingTeam_Player5_Name,VisitingTeam_Player5_Position,VisitingTeam_Player6_ID,VisitingTeam_Player6_Name,VisitingTeam_Player6_Position,VisitingTeam_Player7_ID,VisitingTeam_Player7_Name,VisitingTeam_Player7_Position,VisitingTeam_Player8_ID,VisitingTeam_Player8_Name,VisitingTeam_Player8_Position,VisitingTeam_Player9_ID,VisitingTeam_Player9_Name,VisitingTeam_Player9_Position,HomeTeam_Player1_ID,HomeTeam_Player1_Name,HomeTeam_Player1_Position,HomeTeam_Player2_ID,HomeTeam_Player2_Name,HomeTeam_Player2_Position,HomeTeam_Player3_ID,HomeTeam_Player3_Name,HomeTeam_Player3_Position,HomeTeam_Player4_ID,HomeTeam_Player4_Name,HomeTeam_Player4_Position,HomeTeam_Player5_ID,HomeTeam_Player5_Name,HomeTeam_Player5_Position,HomeTeam_Player6_ID,HomeTeam_Player6_Name,HomeTeam_Player6_Position,HomeTeam_Player7_ID,HomeTeam_Player7_Name,HomeTeam_Player7_Position,HomeTeam_Player8_ID,HomeTeam_Player8_Name,HomeTeam_Player8_Position,HomeTeam_Player9_ID,HomeTeam_Player9_Name,HomeTeam_Player9_Position,Additional_Information,Acquisition_Information
46269,20190405,0,Fri,SEA,AL,9,CHA,AL,6,8,10,51,D,NaN,NaN,NaN,CHI12,32723.0,216,101204000,33000031x,3

In [ ]:
# sched_2023 = pd.read_csv('./data/2023_MLBSchedule.csv')
# existing problems:
# 1. need to process game_time to DayNight
# 2. no indicator for number of games
# 3. some other missing columsn, e.g. HomeTeamLeague, VisitingTeamLeague
sched_2023 = pd.read_csv('./data/2023_schedule_parkid_DN.csv').drop(columns='Unnamed: 0') #added NumberofGames
sched_2023.head()

,Date,game_time,stadium_name,HomeTeam,VisitingTeam,BallParkID,NumberofGames,cnt
0,20230330,2023-03-13 13:05:00,Yankee Stadium,NYA,SFN,CHI11,0.0,1
1,20230330,2023-03-13 13:05:00,Nationals Park,WAS,ATL,WIL02,0.0,1
2,20230330,2023-03-13 13:10:00,Petco Park,SDN,COL,SAN02,0.0,1
3,20230330,2023-03-13 13:20:00,Wrigley Field,CHN,MIL,ATL03,0.0,1
4,20230330,2023-03-13 14:10:00,Fenway Park,BOS,BAL,BOS07,0.0,1


In [ ]:
# concat 2023 schedule to gamelogs data
#sched_2023 = sched_2023.rename(columns={'game_date':'Date', 'home_team':'HomeTeam', 'away_team':'VisitingTeam'})
#gl = pd.concat([gl, sched_2023[['Date','HomeTeam','VisitingTeam']]])
gl = pd.concat([gl, sched_2023[['Date','HomeTeam','VisitingTeam',"BallParkID","NumberofGames"]]])

In [ ]:
def change_abbreviation(df,column_name):
  """change all team names to gamelogs.csv version"""
  replacements = {
      "SF":"SFN",
      "SD":"SDN",
      "TB":"TBA",
      "KC":"KCA",
      "SDP":"SDN",
      "SFG":"SFN",
      "TBR":"TBA",
      "KCR":"KCA",
      "NYY":"NYA",
      "NYM":"NYN",
      "LAD":"LAN",
      "LAA":"ANA",
      "CHC":"CHN",
      "CHW":"CHA",
      "FLA":"MIA",
      "STL":"SLN",
      "WSN":"WAS",
      "MON":"WAS",
      "FLO":"MIA",
  }
  df[column_name]=df[column_name].replace(replacements, regex=False)
  return df

## Date and Time
Create all time related features to the current game, including:


*   Year/month/week/if weekend of the game date: `year`, `month`, `week`, `is_weekend`
*   If it is a public holiday 0/1: `is_public_holiday`
*   Public holiday name: a series of dummy variables
*   If on the first day of a season: `opening_day`

In [ ]:
# add date and time features based on given game date
gl.Date = pd.to_datetime(gl.Date, format='%Y%m%d')
gl['year'] = gl.Date.dt.year
gl['month'] = gl.Date.dt.month
gl['week'] = gl.Date.dt.week
gl['DayofWeek'] = gl.Date.dt.dayofweek
gl['is_weekend'] = np.where(gl.Date.dt.dayofweek>=5,1,0)

In [ ]:
# calculate opening day dummy variable
opening_day = gl.groupby('year').min()['Date'].reset_index()
opening_day['opening_day'] = 1
opening_day.drop(columns='year', inplace=True)

# merge back to gl
gl = gl.merge(opening_day, on='Date', how='left')
gl.opening_day = gl.opening_day.fillna(0)

In [ ]:
# add public holiday information
hol_df = pd.read_csv('./data/us_public_holiday.csv')
hol_df.date = pd.to_datetime(hol_df.date, format='%Y-%m-%d')
hol_df['is_public_holiday'] = 1
gl = gl.merge(hol_df[['date','is_public_holiday','holidayName']], left_on='Date', right_on='date', how='left')
gl.holidayName = gl.holidayName.str.replace('Independence Day (Observed)', 'Independence Day', regex=False)
gl.drop(columns=['date'], inplace=True)

## Uniform team full name and abbreviation to gamelogs.csv version

In [ ]:
gl['HomeTeam']=gl['HomeTeam'].str.replace("MON","WAS")
gl['HomeTeam']=gl['HomeTeam'].str.replace("FLO","MIA")
gl['VisitingTeam']=gl['VisitingTeam'].str.replace("MON","WAS")
gl['VisitingTeam']=gl['VisitingTeam'].str.replace("FLO","MIA")

In [ ]:
location=pd.read_excel("./data/Team_name_location.xlsx")

In [ ]:
location=change_abbreviation(location,"abbreviation")

In [ ]:
location['abbreviation']=np.where(location['team_name']=="Toronto Blue Jays","TOR",location['abbreviation'])

In [ ]:
location=location[location["team_name"] != "Montreal Expos"]

In [ ]:
gl=gl.merge(location,left_on="HomeTeam",right_on="abbreviation",how="left")
gl=gl.rename(columns={"team_name":"HomeTeam_Name","city":"HomeTeam_City","state":"HomeTeam_State"})

In [ ]:
gl=gl.merge(location,left_on="VisitingTeam",right_on="abbreviation",how="left")
gl=gl.rename(columns={"team_name":"VisitingTeam_Name","city":"VisitingTeam_City","state":"VisitingTeam_State"})

In [ ]:
ws_winner=pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/WorldSeries_Winner.csv")
mvp=pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/League_MVPs.csv")
Cy_young=pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/Cy_young_award.csv")

In [ ]:
ws_winner['World_Series_Winner']=np.where(ws_winner['Wins']>ws_winner['Wins.1'],ws_winner['AL Winner'],ws_winner["NL Winner"])
ws_winner['next_year']=ws_winner['Year']+1
gl=gl.merge(ws_winner[['World_Series_Winner',"next_year"]],left_on="year",right_on="next_year")
gl['HomeTeam_ws_winner']=np.where(gl['HomeTeam_Name']==gl['World_Series_Winner'],1,0)
gl['VisitingTeam_ws_winner']=np.where(gl['VisitingTeam_Name']==gl['World_Series_Winner'],1,0)
gl['HomeTeam_ws_winner'].value_counts()

0    54962
1     1813
Name: HomeTeam_ws_winner, dtype: int64

In [ ]:
gl.HomeTeam.unique()

In [ ]:
gl.drop(columns=['next_year',"abbreviation_y"],inplace=True)

In [ ]:
mvp=change_abbreviation(mvp,"Tm")
visiting_name_list=['VisitingTeam_Player1_Name', 'VisitingTeam_Player2_Name', 'VisitingTeam_Player3_Name',
           'VisitingTeam_Player4_Name', 'VisitingTeam_Player5_Name', 'VisitingTeam_Player6_Name', 
           'VisitingTeam_Player7_Name', 'VisitingTeam_Player8_Name', 'VisitingTeam_Player9_Name',"year"]
home_name_list=['HomeTeam_Player1_Name', 'HomeTeam_Player2_Name', 'HomeTeam_Player3_Name',
                    'HomeTeam_Player4_Name', 'HomeTeam_Player5_Name', 'HomeTeam_Player6_Name', 
                    'HomeTeam_Player7_Name', 'HomeTeam_Player8_Name', 'HomeTeam_Player9_Name',"year"]
def look_for_mvp(gl,mvp):
    def contains_name(row):
        names = mvp[mvp['Year']<float(row[-1])]['Name'].tolist()
        for col in row.index:
            if row[col] in names:
                return True
        return False

    gl['MVP_in_visitingteam']=gl[visiting_name_list].apply(contains_name, axis=1)
    gl['MVP_in_hometeam']=gl[home_name_list].apply(contains_name, axis=1)
    return gl

gl=look_for_mvp(gl,mvp)

In [ ]:
def look_for_cy_young(gl,cy_young):
    def contains_name(row):
        names = cy_young[cy_young['Year']==float(row[-1])-1]['Name'].tolist()
        for col in row.index:
            if row[col] in names:
                return True
        return False

    gl['Cy_Young_in_visitingteam']=gl[visiting_name_list].apply(contains_name, axis=1)
    gl['Cy_Young_in_hometeam']=gl[home_name_list].apply(contains_name, axis=1)
    return gl

gl=look_for_cy_young(gl,Cy_young)

In [ ]:
gl.HomeTeam.nunique()

30

## Baseball reference data features
### static and dynamic
**approach**
create primary id to join back to gamelogs data

**static model features**

Win_or_Loss
W_L_ratio
Rank
RA
RS
gb
streak

**dynamic model features**

last_game_div_rank
last_game_w_l_ratio
last_ten_game_w_l_ratio
last_game_record
last_game_gb
last_game_streak
runs_ma
runs_allowed_ma

In [ ]:

# create unique identifier for each game
gl['id'] = gl['Date']+'_' +gl['HomeTeam']+'_'+ gl['VisitingTeam']+'_'+ gl['NumberofGames'].astype('int32').astype('string')

In [ ]:
# read in baseball_reference data
baseball_reference = pd.read_csv('./data/sports_reference_data/mlb-game-data-2000-2022_all.csv')
# some team name are mistakenly coded as 4-letter, e.g. 'TBAD', 'KCAR'
baseball_reference['team'] = baseball_reference['team'].str[:3]
baseball_reference['opponent'] = baseball_reference['opponent'].str[:3]
# align team name abbr.
baseball_reference=change_abbreviation(baseball_reference,'team')
baseball_reference=change_abbreviation(baseball_reference,'opponent')

In [ ]:
# create unique identifier for each game - baseball_reference
  # 1. create partial_id
# baseball_reference['date'] = pd.to_datetime(baseball_reference['date'])
baseball_reference['partial_id'] = baseball_reference['date']+'_'+baseball_reference['team']+'_'+ \
                            baseball_reference['opponent']+'_'
baseball_reference['numberofgames'] = baseball_reference.groupby(['partial_id']).cumcount()+1

  # 2. create numberofgames for baseball reference record
duplicate_game = baseball_reference.groupby(['partial_id'])['date'].count().reset_index().rename(columns = {"date":"duplicate_flag"})
baseball_reference = baseball_reference.merge(duplicate_game, on='partial_id', how='left')
new_num_of_games = np.where(baseball_reference['duplicate_flag']==2,baseball_reference['numberofgames'],0)
baseball_reference['numberofgames'] = new_num_of_games

  # 3. join partial_id and numberofgames
baseball_reference['id'] = baseball_reference['partial_id'] + baseball_reference['numberofgames'].astype('string')

In [ ]:
# merge static and dynamic model features
b_ref_static_model_cols = ['w_or_l_int','w_l_ratio', 'div_rank', 'gb', 'streak', 'runs', 'runs_allowed']
b_ref_dynamic_model_cols = ['last_game_record','last_game_w_l_ratio', 'last_game_div_rank', 'last_game_gb',
                            'last_game_streak', 'runs_ma', 'runs_allowed_ma','last_ten_game_w_l_ratio']
gl = gl.merge(baseball_reference[['id']+b_ref_static_model_cols+b_ref_dynamic_model_cols], on='id', how='left')

## Baseball reference data features
### both model features

***both* feature list**

season_end_rank
season_end_w_l_ratio
season_end_runs_mean
season_end_runs_allowed_mean

In [ ]:
# still need to shift these columns to get the last season data
b_ref_both_model_cols = ['season_end_rank', 'season_end_w_l_ratio','season_end_runs_mean', 'season_end_runs_allowed_mean']

In [ ]:
# initiate season end terms for 2023
# baseball_reference.sort_values(by=['team','date'], inplace=True)
baseball_reference.date = pd.to_datetime(baseball_reference.date)
baseball_reference['year'] = baseball_reference.date.dt.year

season_end_terms_2023 = pd.DataFrame({'team':baseball_reference.team.unique(),
                                      'year':2023})
baseball_reference = pd.concat([baseball_reference, season_end_terms_2023])
# create lag season end terms for 2002 until 2023
season_end_terms = baseball_reference.groupby(['team','year']).last()[b_ref_both_model_cols]
lag_season_end_terms = season_end_terms.groupby('team').shift()
lag_season_end_terms = lag_season_end_terms.reset_index()

In [ ]:
lag_season_end_terms

,team,year,season_end_rank,season_end_w_l_ratio,season_end_runs_mean,season_end_runs_allowed_mean
0,ANA,2000,NaN,NaN,NaN,NaN
1,ANA,2001,3.0,0.506173,5.33,5.36
2,ANA,2002,3.0,0.462963,4.27,4.51
3,ANA,2003,2.0,0.611111,5.25,3.98
4,ANA,2004,3.0,0.475309,4.54,4.59
...,...,...,...,...,...,...
701,WAS,2019,2.0,0.506173,4.76,4.21
702,WAS,2020,2.0,0.574074,5.39,4.47
703,WAS,2021,4.0,0.433333,4.88,5.02
704,WAS,2022,5.0,0.401235,4.47,5.06


In [ ]:
# merge both model features to home team
gl = gl.merge(lag_season_end_terms, left_on=['HomeTeam','year'], right_on=['team','year'], how='left')
gl = gl.drop(columns='team')

In [ ]:
col_dict = {col: "opponent_"+col for col in lag_season_end_terms.columns[2:]}
opponent_lag_season_end_terms = lag_season_end_terms.rename(columns=col_dict)

In [ ]:
# merge both model features to visiting team
gl = gl.merge(opponent_lag_season_end_terms, left_on=['VisitingTeam','year'], right_on=['team','year'], how='left')
gl = gl.drop(columns='team')

## Add other league game schedule

In [ ]:
nfl = pd.read_csv('./data/sports_reference_data/nfl-game-data-2000-2023.csv')
nba = pd.read_csv('./data/sports_reference_data/nba-game-data-2000-2023.csv')
nhl = pd.read_csv('./data/sports_reference_data/nhl-game-data-2000-2023.csv')

In [ ]:
def merge_other_sports(nba,nhl,nfl,gamelog):
    nba_1 = nba[['date','city','state','NBA_Team']] 
    nfl_1 = nfl[['Date','city','state','NFL_team']]
    nhl_1 = nhl[['date','city','state','NHL_Team']]
    
    nba_1 = nba_1.groupby(['date','city','state',])[['NBA_Team']].count()
    nfl_1 = nfl_1.groupby(['Date','city','state',])[['NFL_team']].count()
    nhl_1 = nhl_1.groupby(['date','city','state',])[['NHL_Team']].count()

    nba_1 = nba_1.reset_index()
    nfl_1 = nfl_1.reset_index()
    nhl_1 = nhl_1.reset_index()

    gamelog['HomeTeam_City'] = gamelog['HomeTeam_City'].replace('Washington', 'Washington D.C.')
    gamelog['HomeTeam_State'] = gamelog['HomeTeam_State'].replace('D.C.', 'Washington D.C.')

    gamelog.Date = pd.to_datetime(gamelog.Date)
    nba_1.date = pd.to_datetime(nba_1.date)
    nfl_1.Date = pd.to_datetime(nfl_1.Date)
    nhl_1.date = pd.to_datetime(nhl_1.date)
    
    merged_df = pd.merge(gamelog, nba_1, how='left', left_on=['Date', 'HomeTeam_City','HomeTeam_State'], right_on=['date', 'city','state'])
    merged_df = pd.merge(merged_df, nfl_1, how='left', left_on=['Date', 'HomeTeam_City','HomeTeam_State'], right_on=['Date', 'city','state'])
    merged_df = pd.merge(merged_df, nhl_1, how='left', left_on=['Date', 'HomeTeam_City','HomeTeam_State'], right_on=['date', 'city','state'])
    
    merged_df['NBA_Team'].fillna(0,inplace=True)
    merged_df['NFL_team'].fillna(0,inplace=True)
    merged_df['NHL_Team'].fillna(0,inplace=True)

    merged_df['NBA_Team'] = merged_df['NBA_Team'].astype(int)
    merged_df['NFL_team'] = merged_df['NFL_team'].astype(int)
    merged_df['NHL_Team'] = merged_df['NHL_Team'].astype(int)

    merged_df['NBA_Team'] = merged_df['NBA_Team'].replace(to_replace=range(1, merged_df['NBA_Team'].max()+1), value=1)
    merged_df['NFL_team'] = merged_df['NFL_team'].replace(to_replace=range(1, merged_df['NFL_team'].max()+1), value=1)
    merged_df['NHL_Team'] = merged_df['NHL_Team'].replace(to_replace=range(1, merged_df['NHL_Team'].max()+1), value=1)
    
    merged_df = merged_df.drop(columns=['date_x','city_x','state_x','date_y','city_y','state_y','city','state'])
    
    merged_df = merged_df.rename(columns = {'NFL_team':'NFL_Team'})
    return merged_df
    

In [ ]:
gl = merge_other_sports(nba,nhl,nfl,gl)

## Team player feature
**Number of All-Star players for home/visiting team**

Home_allstar_cnt, Visiting_allstar_cnt

**Average age of home team batting/pitching players**

bat_avg_age, pitch_avg_age

In [ ]:

bat = pd.read_csv('./data/mlb-batting-data-2000-2022.csv').iloc[:,:3]
pit = pd.read_csv('./data/mlb-pitching-data-2000-2022.csv').iloc[:,:3]
as_cnt = pd.read_excel('./data/Allstar cnt.xlsx')

In [ ]:
def avg_age(df, bat, pitch):
    df = df.merge(bat, how='left', left_on=['HomeTeam', 'year'], right_on = ['Teams', 'Year']).drop(['Teams','Year'],axis=1)
    df = df.merge(pitch, how='left', left_on=['HomeTeam', 'year'], right_on = ['Teams', 'Year']).drop(['Teams','Year'],axis=1)
    return df

In [ ]:
def as_count(df,df_as):
    df = df.merge(df_as, how='left', left_on=['HomeTeam', 'year'], right_on = ['teams', 'years']).drop(['teams','years'],axis=1)
    df.rename(columns = {'cnt': 'home_as_cnt'}, inplace=True)
    df = df.merge(df_as, how='left', left_on=['VisitingTeam', 'year'], right_on = ['teams', 'years']).drop(['teams','years'],axis=1)
    df.rename(columns = {'cnt': 'visiting_as_cnt'}, inplace=True)

    df_as['next_year'] = df_as['years']+1
    df = df.merge(df_as, how='left', left_on=['HomeTeam', 'year'], right_on = ['teams', 'next_year']).drop(['teams','years','next_year'],axis=1)
    df.rename(columns = {'cnt': 'previous_home_as_cnt'}, inplace=True)
    df = df.merge(df_as, how='left', left_on=['VisitingTeam', 'year'], right_on = ['teams', 'next_year']).drop(['teams','years','next_year'],axis=1)
    df.rename(columns = {'cnt': 'previous_visiting_as_cnt'}, inplace=True)    
    return df

In [ ]:
gl = avg_age(gl, bat, pit)

In [ ]:
gl = as_count(gl, as_cnt)

# capacity & top10 player cnt

In [ ]:


ca = pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/stadium_capacity.csv")
b = pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/batT10.csv")
f = pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/fieldT10.csv")
p = pd.read_csv("/content/drive/Shareddrives/MinneMUDAC/data/pitchT10.csv")

# capacity
gl = gl.merge(ca[["parkID","Capacity"]], left_on = "BallParkID", right_on = "parkID", how = "left")\
.drop(columns = ["parkID"], axis = 1)

# top10 player cnt
gl = gl.merge(p[["nxtYear","teamID",'pitch10']], left_on = ["year","HomeTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'pitch10':'home_pitch10'})\
.merge(b[["nxtYear","teamID",'bat10']], left_on = ["year","HomeTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'bat10':'home_bat10'})\
.merge(f[["nxtYear","teamID",'field10']], left_on = ["year","HomeTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'field10':'home_field10'})\
.merge(p[["nxtYear","teamID",'pitch10']], left_on = ["year","VisitingTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'pitch10':'visiting_pitch10'})\
.merge(b[["nxtYear","teamID",'bat10']], left_on = ["year","VisitingTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'bat10':'visiting_bat10'})\
.merge(f[["nxtYear","teamID",'field10']], left_on = ["year","VisitingTeam"], right_on = ["nxtYear","teamID"], how = "left")\
.drop(columns = ["nxtYear","teamID"], axis = 1).rename(columns={'field10':'visiting_field10'})

In [ ]:
# remove year 2020, 2021
gl = gl[~gl.year.isin([2020,2021])]

In [ ]:
# add lag attendance
gl = gl.sort_values(by=['HomeTeam','Date','NumberofGames']).reset_index().drop(columns='index')
for lag in [1,2,3,4,5]:
  gl['Attendance_'+str(lag)]=gl.groupby(['HomeTeam'])['Attendance'].shift(lag)

In [ ]:
gl.to_csv("./data/final_gl_Lan.csv", index=False)

In [ ]:
def clean_df(df):
  # drop 2020, 2021, 2023
  df.Date = pd.to_datetime(df['Date'])
  df_model = df[(df['Date']<'2020-01-01') | ((df['Date']>'2021-12-31') & (df['Date']<'2023-01-01'))]
  df_model = df_model.sort_values('Date')

  # Convert the 'day_of_week' column to a category column with custom labels
  category_labels = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
  df_model['DayofWeek'] = df_model['DayofWeek'].astype('category').cat.rename_categories(category_labels)

  # keep the columns can be used as 
  df_model = df_model.iloc[:,list(range(0,13))+list(range(17,18))+list(range(19,77))+list(range(164,168))+list(range(176,182))+list(range(198,214))]

  # drop the columns which value does not make sense 
  df_model = df_model.drop(columns=['VisitingTeam_LineScore','HomeTeam_LineScore'])

  # drop and fill Null values 
  df_model = df_model.dropna(subset=['Attendance','Capacity'])
  df_model['holidayName'] = df_model['holidayName'].fillna('no holiday')
  df_model.is_public_holiday = df_model.is_public_holiday.fillna(0)

  # create Year, Month, DayofMonth time factor 
  df_model['Year'] = df_model.Date.dt.year
  df_model['Month'] = df_model.Date.dt.month
  df_model['DayofMonth'] = df_model.Date.dt.day

  # convert columns to category 
  columns_to_category = ['DayofWeek','VisitingTeam','VisitingTeamLeague','HomeTeam','HomeTeamLeague','DayNight','holidayName',
                       'MVP_in_visitingteam','MVP_in_hometeam','Cy_Young_in_visitingteam','Cy_Young_in_hometeam',
                       'Year','Month','DayofMonth']
  df_model[columns_to_category] = df_model[columns_to_category].astype('category')
  return df_model

In [ ]:
# print na values
gl1 = gl[~gl.year.isin([2023])]
nulls = gl1.isnull().sum()/len(gl1)
nulls = nulls.reset_index().rename(columns={"index":"Column", 0:"Percentage of Null Values"})
nulls.sort_values(by="Percentage of Null Values", ascending=False, inplace=True)

display(HTML(nulls[nulls["Percentage of Null Values"]>0].to_html()))

,Column,Percentage of Null Values
14,Forfeit_Information,1.000000
85,LFUmp_ID,0.999882
87,RFUmp_ID,0.999882
15,Protest_Information,0.999510
13,Completition_Information,0.999353
159,Additional_Information,0.994943
166,is_public_holiday,0.981850
167,holidayName,0.981850
97,SavingPitcher_ID,0.495864
197,last_ten_game_w_l_ratio,0.094535


## JOIN FEATURE IMPORTANCE

In [ ]:
team_df = pd.read_csv('./Model/team_feature_importance_new/feature_importance_TOR.csv')
team_df['Rank'] = team_df['Value'].rank(method='min', ascending=False)

In [ ]:
team_df

,Value,Feature,Rank
0,0,Capacity,92.0
1,0,Cy_Young_in_hometeam,92.0
2,0,Cy_Young_in_visitingteam,92.0
3,0,HomeTeam,92.0
4,0,HomeTeamDefense_PutOuts,92.0
...,...,...,...
112,574,Attendance_4,5.0
113,610,Attendance_3,4.0
114,617,Attendance_2,3.0
115,648,Attendance_5,2.0


In [ ]:
import os

path = './Model/team_feature_importance_new'

# Get all files in the directory
files = os.listdir(path)

all_team_df = pd.DataFrame([])
# Print the file names
for file in files:
    team_df = pd.read_csv(path+'/'+file)
    team_name = file[-7:-4]
    team_df['team_name'] = team_name
    team_df['Rank'] = team_df['Value'].rank(method='min', ascending=False)
    all_team_df = pd.concat([all_team_df, team_df])
    # print(path+'/'+file, team_name)
all_team_df.to_csv('./Model/all_team_feature_importance_and_rank.csv', index=False)

In [ ]:
season_end_performance = gl[(gl.year>=2019)&(gl.year<=2023)].groupby(['HomeTeam','year'])[['season_end_rank','season_end_w_l_ratio']].max().reset_index()

# Wide format results
season_end_performance = season_end_performance.pivot_table(index='HomeTeam',columns='year')

# flatten columns
flat_cols = [str(col[1])+"_"+col[0] for col in season_end_performance.columns]
season_end_performance.columns = flat_cols
season_end_performance = season_end_performance.reset_index()

# season_end_performance.head()

,HomeTeam,2019_season_end_rank,2022_season_end_rank,2023_season_end_rank,2019_season_end_w_l_ratio,2022_season_end_w_l_ratio,2023_season_end_w_l_ratio
0,ANA,4.0,4.0,3.0,0.493827,0.475309,0.450617
1,ARI,3.0,5.0,4.0,0.506173,0.320988,0.456790
2,ATL,1.0,1.0,1.0,0.555556,0.546584,0.623457
3,BAL,5.0,5.0,4.0,0.290123,0.320988,0.512346
4,BOS,1.0,2.0,5.0,0.666667,0.567901,0.481481


In [ ]:
attendance_stats = gl[(gl.year>=2018)&(gl.year<=2022)].groupby('HomeTeam')['Attendance'].agg(['mean','std']).reset_index()
attendance_stats.rename({'mean':'3yr_attedance_mean', 'std':'3yr_attedance_std'},inplace=True)

In [ ]:
cluster_df = season_end_performance.merge(attendance_stats, on='HomeTeam')

In [ ]:
from sklearn.cluster import KMeans

# Initialize the KMeans algorithm with 5 clusters
kmeans = KMeans(n_clusters=4)

cluster_df.fillna(0,inplace=True)
# Fit the KMeans model to the data
kmeans.fit(cluster_df[cluster_df.columns[1:]])

# Get the cluster labels for each point
labels = kmeans.labels_

# Get the centroids of each cluster
centroids = kmeans.cluster_centers_

KMeans(n_clusters=4)

In [ ]:
pd.concat([pd.DataFrame([0,1,2,3]), pd.DataFrame(centroids)], axis=1)
# 3, 1, 2, 0

,0,0,1,2,3,4,5,6,7
0,0,3.700000,3.100000,3.600000,0.445618,0.477160,0.448148,17668.289552,7121.655888
1,1,2.000000,1.750000,2.750000,0.566566,0.482675,0.519645,34881.944634,5944.610448
2,2,3.444444,3.777778,2.888889,0.475995,0.458848,0.503429,27523.274942,8071.476088
3,3,2.000000,2.000000,1.000000,0.574970,0.592593,0.623457,43584.761289,5371.714718


In [ ]:
cluster_df.columns

Index(['HomeTeam', '2019_season_end_rank', '2022_season_end_rank',
       '2023_season_end_rank', '2019_season_end_w_l_ratio',
       '2022_season_end_w_l_ratio', '2023_season_end_w_l_ratio', 'mean',
       'std'],
      dtype='object')

In [ ]:
cluster_df['label'] = labels

In [ ]:
cluster_df.sort_values('label')

,HomeTeam,2019_season_end_rank,2022_season_end_rank,2023_season_end_rank,2019_season_end_w_l_ratio,2022_season_end_w_l_ratio,2023_season_end_w_l_ratio,mean,std,label
14,MIA,5.0,4.0,4.0,0.391304,0.413580,0.425926,10411.082305,4730.365160,0
26,TBA,3.0,1.0,3.0,0.555556,0.617284,0.530864,14246.234568,5067.201214,0
3,BAL,5.0,5.0,4.0,0.290123,0.320988,0.512346,17450.065844,8338.120071,0
21,PIT,4.0,5.0,4.0,0.509317,0.376543,0.382716,17414.103306,7502.056049,0
5,CHA,4.0,1.0,2.0,0.382716,0.574074,0.500000,21768.392562,7485.563600,0
19,OAK,2.0,3.0,5.0,0.598765,0.530864,0.370370,16662.198347,10205.119787,0
7,CIN,5.0,3.0,4.0,0.413580,0.512346,0.382716,19893.831276,7756.593892,0
8,CLE,1.0,2.0,1.0,0.561728,0.493827,0.567901,20416.514403,7970.979422,0
10,DET,3.0,3.0,4.0,0.395062,0.475309,0.407407,20221.081967,6483.098224,0
12,KCA,5.0,4.0,5.0,0.358025,0.456790,0.401235,18199.390947,5677.461463,0
